In [2]:
%matplotlib qt
import hyperspy.api as hs
import pyxem as pxm
import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import blob_log
from pathlib import Path
from diffsims.utils.sim_utils import get_electron_wavelength

# Dataset A

## Load dataset
Load the dataset, slice it to ROI and set the diffraction calibration

In [2]:
filename = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Site2\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3.hspy')

signal = hs.load(str(filename), lazy=True).inav[0:512, 0:512]
signal.set_diffraction_calibration(0.00952) #in Å^-1

ERROR:hyperspy.io:If this file format is supported, please report this error to the HyperSpy developers.


OSError: [Errno 22] Unable to open file (file read failed: time = Mon Feb  6 18:47:54 2023
, filename = 'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Site2\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3.hspy', file descriptor = 4, errno = 22, error message = 'Invalid argument', buf = 0000000BEBBE9B60, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)

## "Correct" linear beam tilt/shift impurity 
(make the direct beam position more stable)

In [ ]:
old_max = signal.max(axis=[0, 1])
old_max.metadata.General.title = 'Before descan correction'
com = signal.center_off_mass(mask=(127, 126, 12.5))
beam_shift = pxm.signals.BeamShift(com.T)
mask = hs.signals.Signal2D(np.zeros(signal.axes_manager.navigation_shape, dtype=bool).T).T
mask.inav[20:-20, 20:-20] = True
beam_shift.make_linear_plane(mask=mask)
beam_shift = beam_shift - (signal.axes_manager.signal_shape[0] //2)
signal.shift_diffraction(beam_shift.isig[0], beam_shift.isig[1])
signal.metadata.add_dictionary({
        'Preprocessing': {
            'Centering': {
                'COM': com,
                'COM_mask': {
                    'x': com_mask[0],
                    'y': com_mask[1],
                    'r': com_mask[2]
                },
                'Shifts': beam_shift,
                'shift_estimate_mask': mask
            }
        }
    })
new_max = signal.max(axis=[0, 1])
new_max.metadata.General.title = 'After descan correction'
hs.plot.plot_images([old_max, new_max], colorbar='single', axes_decor='off')

## Rebin data
Bin the diffraction patterns by 2

In [ ]:
signal.rebin(scale=(1, 1, 2, 2), out=signal)

## Find masks for reflections

In [ ]:
image = signal.mean(axis=[0, 1]).deepcopy() #The image to search for blobs in
a = 4.04
minimum_r = 8 #The minimum radius to set for any mask
cutoff_hkl = np.array([2, 2, 0]) #Make a mask with cutoff at a given HKL
cuton_mrad = 4 #Make a mask that cutsoff everything up a certain mrad

#Set up mask arrays
nx, ny = image.axes_manager.signal_shape
mask = np.zeros((nx, ny), dtype=bool)
direct_beam_mask = np.zeros((nx, ny), dtype=bool)
cutoff_mask = np.zeros((nx, ny), dtype=bool)

#Setting cutoffs
cutoff_g = np.sqrt(np.sum(cutoff_hkl**2 / a**2))
cuton_k = cuton_mrad / 1000 / get_electron_wavelength(image.metadata.Acquisition_instrument.TEM.beam_energy/1000)
print(f'Minimum scattering vector: {cuton_k} {image.axes_manager[0].units}\nMaximum scattering vector: {cutoff_g} {image.axes_manager[0].units}')
X, Y = np.meshgrid(image.axes_manager[0].axis, image.axes_manager[1].axis)
#Set outer cutoff
R = np.sqrt(X**2 + Y**2)
cutoff_mask[R>=cutoff_g] = True
#Set inner cutoff
R = np.sqrt(X**2 + Y**2)
direct_beam_mask[R<=cuton_k] = True

#Mask reflections
blob_kwargs = {
    'min_sigma': 1,
    'max_sigma': 15,
    'num_sigma': 100,
    'overlap': 0,
    'threshold': 1E-18,
}

# Look for blobs (reflections)
blobs = blob_log(image.data, **blob_kwargs)
print(len(blobs))
xs, ys = np.arange(0, nx), np.arange(0, ny)
X, Y = np.meshgrid(xs, ys)
for blob in blobs:
    y, x, r = blob  # x and y axes are flipped in hyperspy compared to numpy
    r = np.sqrt(2) * r  # Scale blob radius to appear more like a real radius
    r = max([minimum_r, r])  # Make sure that the radius is at least the specified minimum radius
    R = np.sqrt((X - x) ** 2 + (Y - y) ** 2)
    mask[R < r] = True

fig, axes = plt.subplots(nrows=1, ncols=6, sharex=True, sharey=True)
axes[0].imshow(image.data)
axes[1].imshow(mask)
axes[2].imshow(direct_beam_mask)
axes[3].imshow(cutoff_mask)
axes[4].imshow((image.data * ~(mask + direct_beam_mask))**0.256)
axes[5].imshow((image.data * ~cutoff_mask)**0.256)

#Create signals
direct_beam_mask = hs.signals.Signal2D(direct_beam_mask)
direct_beam_mask.metadata.General.title = f'>{cuton_k} {signal.axes_manager[-1].units} mask'

cutoff_mask = hs.signals.Signal2D(cutoff_mask)
cutoff_mask.metadata.General.title = f'<{cutoff_g} {signal.axes_manager[-1].units} mask'

mask = hs.signals.Signal2D(mask)
mask.metadata.General.title = f'Reflection mask'
mask.metadata.add_dictionary({'Preprocessing': {'blob_log': blob_kwargs,
                                                'minimum_r': minimum_r}})

for m in [direct_beam_mask, mask, cutoff_mask]:
    for ax in range(image.axes_manager.signal_dimension):
        m.axes_manager[ax].scale = image.axes_manager[ax].scale
        m.axes_manager[ax].units = image.axes_manager[ax].units

# Add metadata
signal.metadata.add_dictionary({
    'Preprocessing': {
        'Masks': {
            'Diffraction': {
                'direct_beam': direct_beam_mask,
                'reflections': mask,
                'cutoff': cutoff_mask
            }
        }
    }
})

## Normalize the data

In [ ]:
signal.change_dtype('float32')
signal = signal / signal.nanmax(axis=[0, 1, 2, 3])

## Save the preprocessed data

In [ ]:
signal.save(filename.with_stem(f'{filename.stem}_preprocessed'))

# Dataset B

## Load dataset
Load the dataset, slice it to ROI and set the diffraction calibration

In [ ]:
filename = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Site2\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3.hspy')

signal = hs.load(str(filename), lazy=True).inav[0:512, 0:512]
signal.set_diffraction_calibration(0.00943) #in Å^-1

## "Correct" linear beam tilt/shift impurity 
(make the direct beam position more stable)

In [ ]:
old_max = signal.max(axis=[0, 1])
old_max.metadata.General.title = 'Before descan correction'
com = signal.center_off_mass(mask=(127, 126, 12.5))
beam_shift = pxm.signals.BeamShift(com.T)
mask = hs.signals.Signal2D(np.zeros(signal.axes_manager.navigation_shape, dtype=bool).T).T
mask.inav[20:-20, 20:-20] = True
beam_shift.make_linear_plane(mask=mask)
beam_shift = beam_shift - (signal.axes_manager.signal_shape[0] //2)
signal.shift_diffraction(beam_shift.isig[0], beam_shift.isig[1])
signal.metadata.add_dictionary({
        'Preprocessing': {
            'Centering': {
                'COM': com,
                'COM_mask': {
                    'x': com_mask[0],
                    'y': com_mask[1],
                    'r': com_mask[2]
                },
                'Shifts': beam_shift,
                'shift_estimate_mask': mask
            }
        }
    })
new_max = signal.max(axis=[0, 1])
new_max.metadata.General.title = 'After descan correction'
hs.plot.plot_images([old_max, new_max], colorbar='single', axes_decor='off')

## Rebin data
Bin the diffraction patterns by 2

In [ ]:
signal.rebin(scale=(1, 1, 2, 2), out=signal)

## Find masks for reflections

In [ ]:
image = signal.mean(axis=[0, 1]).deepcopy() #The image to search for blobs in
a = 4.04
minimum_r = 8 #The minimum radius to set for any mask
cutoff_hkl = np.array([2, 2, 0]) #Make a mask with cutoff at a given HKL
cuton_mrad = 4 #Make a mask that cutsoff everything up a certain mrad

#Set up mask arrays
nx, ny = image.axes_manager.signal_shape
mask = np.zeros((nx, ny), dtype=bool)
direct_beam_mask = np.zeros((nx, ny), dtype=bool)
cutoff_mask = np.zeros((nx, ny), dtype=bool)

#Setting cutoffs
cutoff_g = np.sqrt(np.sum(cutoff_hkl**2 / a**2))
cuton_k = cuton_mrad / 1000 / get_electron_wavelength(image.metadata.Acquisition_instrument.TEM.beam_energy/1000)
print(f'Minimum scattering vector: {cuton_k} {image.axes_manager[0].units}\nMaximum scattering vector: {cutoff_g} {image.axes_manager[0].units}')
X, Y = np.meshgrid(image.axes_manager[0].axis, image.axes_manager[1].axis)
#Set outer cutoff
R = np.sqrt(X**2 + Y**2)
cutoff_mask[R>=cutoff_g] = True
#Set inner cutoff
R = np.sqrt(X**2 + Y**2)
direct_beam_mask[R<=cuton_k] = True

#Mask reflections
blob_kwargs = {
    'min_sigma': 1,
    'max_sigma': 15,
    'num_sigma': 100,
    'overlap': 0,
    'threshold': 1E-18,
}

# Look for blobs (reflections)
blobs = blob_log(image.data, **blob_kwargs)
print(len(blobs))
xs, ys = np.arange(0, nx), np.arange(0, ny)
X, Y = np.meshgrid(xs, ys)
for blob in blobs:
    y, x, r = blob  # x and y axes are flipped in hyperspy compared to numpy
    r = np.sqrt(2) * r  # Scale blob radius to appear more like a real radius
    r = max([minimum_r, r])  # Make sure that the radius is at least the specified minimum radius
    R = np.sqrt((X - x) ** 2 + (Y - y) ** 2)
    mask[R < r] = True

fig, axes = plt.subplots(nrows=1, ncols=6, sharex=True, sharey=True)
axes[0].imshow(image.data)
axes[1].imshow(mask)
axes[2].imshow(direct_beam_mask)
axes[3].imshow(cutoff_mask)
axes[4].imshow((image.data * ~(mask + direct_beam_mask))**0.256)
axes[5].imshow((image.data * ~cutoff_mask)**0.256)

#Create signals
direct_beam_mask = hs.signals.Signal2D(direct_beam_mask)
direct_beam_mask.metadata.General.title = f'>{cuton_k} {signal.axes_manager[-1].units} mask'

cutoff_mask = hs.signals.Signal2D(cutoff_mask)
cutoff_mask.metadata.General.title = f'<{cutoff_g} {signal.axes_manager[-1].units} mask'

mask = hs.signals.Signal2D(mask)
mask.metadata.General.title = f'Reflection mask'
mask.metadata.add_dictionary({'Preprocessing': {'blob_log': blob_kwargs,
                                                'minimum_r': minimum_r}})

for m in [direct_beam_mask, mask, cutoff_mask]:
    for ax in range(image.axes_manager.signal_dimension):
        m.axes_manager[ax].scale = image.axes_manager[ax].scale
        m.axes_manager[ax].units = image.axes_manager[ax].units

# Add metadata
signal.metadata.add_dictionary({
    'Preprocessing': {
        'Masks': {
            'Diffraction': {
                'direct_beam': direct_beam_mask,
                'reflections': mask,
                'cutoff': cutoff_mask
            }
        }
    }
})

## Normalize the data

In [ ]:
signal.change_dtype('float32')
signal = signal / signal.nanmax(axis=[0, 1, 2, 3])

## Save the preprocessed data

In [ ]:
signal.save(filename.with_stem(f'{filename.stem}_preprocessed'))